In [71]:
import pandas as pd
import networkx as nx
import openpyxl
import matplotlib.pyplot as plt

In [83]:
# 単価テーブル
unit_price = pd.read_excel('ブレーカ調査/CT単価表.xlsx', sheet_name='Sheet1')

In [84]:
# 案2_主幹第1レイヤーのみ
case_2 = pd.read_excel('ブレーカ調査/241211_ブレーカ調査_トップレイヤーのみ.xlsx', sheet_name='Sheet1', header=1)

In [85]:
# 必要CT数
line_num = pd.read_excel('ブレーカ調査/必要CT数.xlsx', sheet_name='Sheet1')

In [86]:
case_2.columns

Index(['No.', '建屋', '階', '部屋名', '部署', '分電盤\nNo.', '分電盤名', '親分電盤\n(あれば)',
       '電力量計\nあり/なし', 'ﾌﾞﾚｰｶ番号\n(あれば)', '電圧\n200/100', '相\n3/1', 'ﾌﾞﾚｰｶ容量\nA',
       '負荷名', 'ｹｰﾌﾞﾙ径\nmm', '備考', '電力量計\n取付対象', '線', '型番', '単価', '必要CT数', '小計',
       '電力計', '電力計単価', 'M5Stack', 'M5Stack単価'],
      dtype='object')

必要CT数とCT代計算

In [87]:
for index, row in case_2.iterrows():
    volt_str = row['電圧\n200/100']
    phase_str = row['相\n3/1']
    
    try:
        volt = int(volt_str)
    except:
        if volt_str == '200/100':
            volt = 200
        else:
            volt = None
    
    try:
        phase = int(phase_str)
    except:
        phase = None
    
    if volt == 100 or volt_str == '100/200':
        case_2.at[index, '線'] = 2
    elif volt == 200 and phase == 3:
        case_2.at[index, '線'] = 3
    elif volt == 200 and phase == 1:
        case_2.at[index, '線'] = 2
    else:
        case_2.at[index, '線'] = 3

In [88]:
for index, row in case_2.iterrows():
    if row['電力量計\n取付対象']==1:
        volt_str = row['電圧\n200/100']
        try:
            volt = int(volt_str)
        except:
            if volt_str == '200/100':
                volt = 200
            else:
                volt = None
        phase_str = row['相\n3/1']
        try:
            phase = int(phase_str)
        except:
            phase = None
        current_str = row['ﾌﾞﾚｰｶ容量\nA']
        try:
            current = int(current_str)
        except:
            current = None
        diameter_str = row['ｹｰﾌﾞﾙ径\nmm']
        try:
            diameter = int(diameter_str)
        except:
            diameter = None
        line = int(row['線'])
        
        filtered = unit_price[(unit_price['最大線径[mm]'] >= diameter) & (unit_price['1次側定格電流[A]'] >= current)]
        if not filtered.empty:
            case_2.at[index,'型番'] = filtered.iloc[0]['型番']
            case_2.at[index,'単価'] = filtered.iloc[0]['単価']
        else:
            print('Not Found:', row['No.'],diameter,current)
            case_2.at[index,'型番'] = None
            case_2.at[index,'単価'] = None
        filtered_ct_num = line_num[(line_num['相'] == phase) & (line_num['線'] == line)]
        if not filtered_ct_num.empty:
            case_2.at[index,'必要CT数'] = filtered_ct_num.iloc[0]['必要CT数']
        else:
            print('Not Found:', row['No.'],phase,line)
            case_2.at[index,'必要CT数'] = None
case_2['小計'] = case_2['単価'] * case_2['必要CT数']



Not Found: nan 150 3


C:\Users\00160428630\AppData\Local\Temp\ipykernel_3596\3658427982.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'KM-NCT-225A' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  case_2.at[index,'型番'] = filtered.iloc[0]['型番']


In [89]:
# 型番ごとの必要CT数と金額を計算
model_ct_sum = case_2.groupby('型番')['必要CT数'].sum()
model_price_sum = case_2.groupby('型番')['小計'].sum()

# 結果を表示
print("型番ごとの必要CT数:")
print(model_ct_sum)
print("\n型番ごとの金額:")
print(model_price_sum)

型番ごとの必要CT数:
型番
KM-NCT-100A     34.0
KM-NCT-225A     61.0
KM-NCT-400A     49.0
KM-NCT-600A     12.0
M-NCT-5A/50A    35.0
Name: 必要CT数, dtype: float64

型番ごとの金額:
型番
KM-NCT-100A     109480.0
KM-NCT-225A     237900.0
KM-NCT-400A     254800.0
KM-NCT-600A      62400.0
M-NCT-5A/50A    100800.0
Name: 小計, dtype: float64


CTはそろったので、必要な電力計数を算出したい

In [90]:
case_2['小計'].sum()

np.float64(765380.0)

In [91]:
case_2['必要CT数'].sum()

np.float64(191.0)

In [92]:
# 型番ごとの必要CT数を再計算
model_ct_sum = case_2.groupby('型番')['必要CT数'].sum()

# 全体の必要CT数を再計算
total_ct_sum = case_2['必要CT数'].sum()

# 結果を表示
print("型番ごとの必要CT数:")
print(model_ct_sum)
print("\n全体の必要CT数:")
print(total_ct_sum)

# 型番ごとの必要CT数の合計と全体の必要CT数が一致するか確認
if model_ct_sum.sum() == total_ct_sum:
    print("\n型番ごとの必要CT数の合計と全体の必要CT数は一致します。")
else:
    print("\n型番ごとの必要CT数の合計と全体の必要CT数は一致しません。")
    print(f"型番ごとの必要CT数の合計: {model_ct_sum.sum()}")
    print(f"全体の必要CT数: {total_ct_sum}")

型番ごとの必要CT数:
型番
KM-NCT-100A     34.0
KM-NCT-225A     61.0
KM-NCT-400A     49.0
KM-NCT-600A     12.0
M-NCT-5A/50A    35.0
Name: 必要CT数, dtype: float64

全体の必要CT数:
191.0

型番ごとの必要CT数の合計と全体の必要CT数は一致します。


In [93]:
case_2.to_csv('case_2.csv', index=False)

In [95]:
case_2.columns

Index(['No.', '建屋', '階', '部屋名', '部署', '分電盤\nNo.', '分電盤名', '親分電盤\n(あれば)',
       '電力量計\nあり/なし', 'ﾌﾞﾚｰｶ番号\n(あれば)', '電圧\n200/100', '相\n3/1', 'ﾌﾞﾚｰｶ容量\nA',
       '負荷名', 'ｹｰﾌﾞﾙ径\nmm', '備考', '電力量計\n取付対象', '線', '型番', '単価', '必要CT数', '小計',
       '電力計', '電力計単価', 'M5Stack', 'M5Stack単価'],
      dtype='object')

In [96]:
# 電力計の連番を振るためのカウンタ
power_meter_counter = 1

# 電力計の連番を格納するリスト
power_meter_list = []

panel_name_unique = case_2['分電盤名'].unique()

# 分電盤ごとに処理を行う
for panel in panel_name_unique:
    print(panel)
    panel_data = case_2[case_2['分電盤名'] == panel].reset_index(drop=True)
    ct_count = 0
    for index, row in panel_data.iterrows():
        if pd.notnull(row['必要CT数']):
            ct_count += row['必要CT数']
            if ct_count <= 4:
                power_meter_list.append(f'電力計_{power_meter_counter}')
            else:
                power_meter_counter += 1
                ct_count = row['必要CT数']
                power_meter_list.append(f'電力計_{power_meter_counter}')
        else:
            power_meter_list.append(None)
    if panel_data['電力量計\n取付対象'].sum() > 0:
        power_meter_counter += 1

    if power_meter_counter == 37:
        print('HIT')
    print(power_meter_counter)

# 電力計の連番をデータフレームに追加
case_2['電力計'] = power_meter_list


1L-1
2
1M-1
4
2L-1
5
3L-1
6
L-1南側外灯電源盤
6
L33-1A
7
L33-1B
8
L33-2A(右)
9
L33-2A(左)
10
L33-2E
11
L33-3(検査)
12
L33-3A
13
M33-1A
14
M33-1B
15
M33-1B-1
16
M33-1F
16
M33-1G
17
M33-1H(右)
18
M33-1H(左)
20
M33-1I
22
M33-1J
23
M33-1J-1(1)
23
M33-1J-1(2)
23
M33-1K
29
M33-1K-1
29
M33-1L
32
M33-1N(A)
33
M33-1N(B)
36
M33-1W
41
M33-1W-1盤
42
M33-2A
43
M33-2F
44
M33-3(再研)
45
M33-31(NT)
46
M33-3-1(再研)
46
M33-3A
47
M33-3B
48
M33-4D
49
ML-2
50
ML-30
51
ML33-4A
52
PAC-1
53
SE-3 A盤
54
SE-3 B盤
55
ﾀﾞｸﾄﾌｧﾝ操作盤
56
ﾌｧﾝ制御盤
56
ﾎﾙﾀﾞ測定室分電盤
56
ﾚｰｽｳｪｲ分電盤
57
ﾜｲﾔｰ加工電源盤
58
局所排気制御盤
58
局所排気設備制御盤
59
局所排気設備制御盤電源盤
60
局所排気装置動力盤
60
空調ｴｱﾊﾝ運転盤
60
空調動力盤
61
工具1F動力M-1(左)
61
工具1F動力M-2(中)
61
工具1F動力M-3(右)
61
工具EHP→工具空調機動力盤
64
工具FAN制御盤
65
工具L-1
66
工具L-2
67
工具M-1
68
工具M-1-2
69
工具ﾚｲｱｳﾄA盤
70
工具ﾚｲｱｳﾄB盤
71
工具電灯関係
72
恒温ﾌﾞｰｽ用動力盤
72
手元開閉盤
72
洗浄機盤
72
電源盤
72
電灯盤L-2
73
電灯盤ML-B.C.D
74
動力盤1M-1-1
74
動力盤A
75
動力盤B
75
動力盤C
75
動力盤D
76
動力分電盤
77
熱源制御盤
78
排気ﾌｧﾝ操作盤
78
排熱ﾀﾞｸﾄ盤
78
盤名なし
78


In [97]:
del case_2['電力計単価']

In [98]:
filled = []
for index,row in case_2.iterrows():
    if pd.notnull(row['電力計']):
        if not row['電力計'] in filled:
            filled.append(row['電力計'])
            case_2.at[index,'電力計単価'] = 11640


In [99]:
int(case_2['小計'].sum())

765380

In [100]:
int(case_2['電力計単価'].sum())

884640

In [101]:
# M5Stackの連番を振るためのカウンタ
m5stack_counter = 1

# M5Stackの連番を格納するリスト
m5stack_list = []

# 分電盤ごとに処理を行う
for panel in case_2['分電盤名'].unique():
    panel_data = case_2[case_2['分電盤名'] == panel]
    power_meter_count = 0
    for index, row in panel_data.iterrows():
        if pd.notnull(row['電力計']):
            power_meter_count += 1
            if power_meter_count > 2:
                m5stack_counter += 1
                power_meter_count = 1
            m5stack_list.append(f'M5Stack_{m5stack_counter}')
        else:
            m5stack_list.append(None)
    if panel_data['電力量計\n取付対象'].sum() > 0:
        m5stack_counter += 1

# M5Stackの連番をデータフレームに追加
case_2['M5Stack'] = m5stack_list

In [102]:
filled = []
for index,row in case_2.iterrows():
    if pd.notnull(row['M5Stack']):
        if not row['M5Stack'] in filled:
            filled.append(row['M5Stack'])
            case_2.at[index,'M5Stack単価'] = 7000

In [103]:
int(case_2['小計'].sum())

765380

In [105]:
int(case_2['電力計単価'].sum())

884640

In [106]:
int(case_2['M5Stack単価'].sum())

539000

In [107]:
case_2.to_excel('ブレーカ調査/CASE2_主幹のみ.xlsx', index=False)